In [1]:
from DefSD import *


In [122]:
#   a table resultListDT relates the name of the objects and the resulting temoerature and density
#   for the four couples of species

resultListDT=Table.read('resultListDT.tbl', format='ascii')
ion = [' [OIII]', ' [SII]', ' [NII]', ' [OII]', ' [NII]', ' [SII]', ' [OIII]', ' [OII]']
for i in range(1,5):
    colT= Column(name='Temp_'+str(i)+ion[2*(i-1)], length=len(resultListDT),unit=u.K)
    colD= Column(name='Den_'+str(i)+ion[2*(i-1)+1], length=len(resultListDT),unit=1 / (u.cm**3))
    resultListDT.add_columns([colT,colD])
resultListDT

#   Obtain the lists Spectra, Headers, spectrum variance of the intensity according with a list of galaxy names
#   and a Table Tabla of models for each line for all the spectra

Headers = []
Spectra = []
SpectraCont = []
Lambda = []
Modelos=[]
Tablas=[]
HDUs=[]
TypeFITS=[]
SpectraIvar = []
LINEProfile = []
paramCont=0.1
typeFITS =0
for x in range(len(resultListDT['ObjID'])):
    name = resultListDT['ObjID'][x]
    z=resultListDT['z'][x]
    isFits = resultListDT['isFits'][x]
    if isFits:
        hdulist = fits.open('Exam_fits/'+name+'.fits')
        head= hdulist[0].header
        typeFITS=0
        if head['NAXIS'] == 0:
            total= hdulist[1].data
        elif head['NAXIS'] == 1:
            typeFITS=1
        if typeFITS==0:
            spectraFlux = total['flux']
            spectraIvar = total['ivar']
            linlam=np.exp(total['loglam']*np.log(10))          #  ocultar para leer desde archivo
            lambd = linlam/(1+z)
        else:
            spectraFlux = hdulist[0].data #*(1e-19)
            spectraIvar = spectraFlux*0
            if 'CDELT1' in head:
                lambd = np.linspace(head['CRVAL1'], head['CRVAL1']+head['CDELT1']*(len(spectraFlux)-1), len(spectraFlux))
            elif 'W0' in head:
                lambd = np.linspace(head['W0'], head['W0']+head['WPC']*(len(spectraFlux)-1), len(spectraFlux))
            else:
                print "Can't identify FITS format in file: ", str(name)
                break
            lambd = lambd/(1+z)
    else:
        specTable = Table.read(str(name)+'.txt', format='ascii')
        spectraFlux = specTable['Intensity'].data #.tolist()
        spectraIvar = spectraFlux*0
        lambd = specTable['Spectral-axis'].data #.tolist()
        lambd = lambd/(1+z)
    orderU = 10**(int(mt.log(np.mean(spectraFlux), 10))-1)
#    scaleFactor = 10**(-int(mt.log(np.mean(spectraFlux), 10))+1)
#    spectraFlux=spectraFlux*scaleFactor
    
    SpectraIvar.append(spectraIvar)
    TypeFITS.append(typeFITS)
    Lambda.append(lambd)
#    HDUs.append(hdulist)
#    Headers.append(head)   #Datos de la cabecera del archivo .fits
    Spectra.append(spectraFlux)
    print "Objeto: " + name
    spec =np.array(spectraFlux)
    spec2, lambd = r(spec, lambd, paramCont*orderU)
    
    
    limResolucion = mt.ceil(lambd[1]-lambd[0])
    
# ..............    
    
    if lambd[0] > 3720.0 :
        print "The object isn't in expected spectrum region"
        ListaLineas=Table.read('LinesPrincipal.txt', format='ascii')
        LS=np.zeros([13,9])
        
        lineProf = lineProfile(x,spec2, lambd,ListaLineas,Spectra, limResolucion)
        LINEProfile.append(lineProf)
        # Este ciclo calcula remplaza la región del contínuo que está justo en el perfil de línea, por una recta
        # que es mas preciso al determinar la amplitud del modelo gaussiano
        for ina in range(len(lineProf)):
            lInf = lineProf[ina]['inf']
            lSup = lineProf[ina]['sup']
            lDel = lSup-lInf
            if (lInf-3*lDel) >= 0 :
                lInf = lInf-3*lDel
            if (lSup+3*lDel) <= len(lambd) :
                lSup = lSup+3*lDel
            m1, b1 = linModel( lambd[lInf] , spec2[lInf] , lambd[lSup], spec2[lSup])
            lineLambd = lambd[lInf:lSup]
            lineIntCont = lines( lineLambd , m1, b1)
            spec2[lInf:lSup] = lineIntCont
        ModelG1, lineProf, spec2 = finalFit(x, spec2, lambd,ListaLineas,Spectra, spectraIvar,2000, limResolucion)
#        ModelL1 = fit2(x, spec2, lambd,ListaLineas,Spectra,2500)
    else:
        # Este ciclo calcula remplaza la región del contínuo que está justo en el perfil de línea, por una recta
        # que es mas preciso al determinar la amplitud del modelo gaussiano
        ListaLineas=Table.read('LinesAll.txt', format='ascii')
        lineProf = lineProfile(x,spec2, lambd,ListaLineas,Spectra, limResolucion)
        LINEProfile.append(lineProf)
        for ina in range(len(lineProf)):
            lInf = lineProf[ina]['inf']
            lSup = lineProf[ina]['sup']
            lDel = lSup-lInf
            if (lInf-3*lDel) >= 0 :
                lInf = lInf-3*lDel
            if (lSup+3*lDel) <= len(lambd) :
                lSup = lSup+3*lDel
            m1, b1 = linModel( lambd[lInf] , spec2[lInf] , lambd[lSup], spec2[lSup])
            lineLambd = lambd[lInf:lSup]
            lineIntCont = lines( lineLambd , m1, b1)
            spec2[lInf:lSup] = lineIntCont
        LS=np.zeros([len(ListaLineas),9])
        ModelG1, lineProf, spec2 = finalFit(x, spec2, lambd,ListaLineas,Spectra, spectraIvar,2000, limResolucion)
#        ModelL1 = fit2(x, spec2, lambd,ListaLineas,Spectra,2500)
    
    Tabla=Table(LS,names=('Amplitude','Wavelength','Sigma','Lambda Vac','Element','Flux','Flux_Err', 'FWHM', 'eqWidth'),
                dtype=('f8','f8','f8','f8','S8','f8','f8','f8','f8'))
    Tabla['Lambda Vac']=ListaLineas['LAMBDA VAC ANG']
    Tabla['Element']=ListaLineas['SPECTRUM']
    AA=split_list(ModelG1[0].parameters, wanted_parts=len(ListaLineas))
    AAA=split_list(ModelG1[1].parameters, wanted_parts=2)
    SpectraCont.append(spec2)
#    AA=split_list(ModelL1[0].parameters, wanted_parts=len(ListaLineas))
#    AAA=split_list(ModelL1[1].parameters, wanted_parts=2)
    for ii in range(len(ListaLineas)):
        if AA[ii][2]<0:
            Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AA[ii][0], AA[ii][1], mt.fabs(AA[ii][2])])
        else:
            Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=AA[ii]
        if ii==0 or ii==1:
            if AAA[ii][2]<0:
                Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AAA[ii][0], AAA[ii][1], mt.fabs(AAA[ii][2])])
            else:
                Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=AAA[ii]
    Modelos.append(ModelG1)
    #    Modelos.append(ModelL1)
    print Tabla 
    Tablas.append(Tabla)

Objeto: SDSS J170918.48+621731.5SP1
It has not been possible to fit the line:  5756.24
It has not been possible to fit the line:  6529.04
  Amplitude     Wavelength      Sigma      Lambda Vac ... Flux_Err FWHM eqWidth
------------- ------------- -------------- ---------- ... -------- ---- -------
23.2194792212 3727.02411685  1.33813822365   3727.092 ...      0.0  0.0     0.0
31.2626588832 3729.76488351  1.14203793096   3729.875 ...      0.0  0.0     0.0
12.3796863207 4341.52826646 0.812842687312   4341.684 ...      0.0  0.0     0.0
2.22288785359 4364.87897644  2.88642072881   4364.436 ...      0.0  0.0     0.0
25.9225838012 4862.52327468  1.31015672897   4862.683 ...      0.0  0.0     0.0
15.8075300236 4945.19665676 0.306547181571   4932.603 ...      0.0  0.0     0.0
15.2379012203 4960.40816747   1.9479364703   4960.295 ...      0.0  0.0     0.0
45.4847459477 5008.09877015  1.99648191965    5008.24 ...      0.0  0.0     0.0
          0.0       5756.24            0.0    5756.24 ...     

ValueError: operands could not be broadcast together with shapes (2,) (3,) 

In [123]:
#   plot of the galaxy with index jj observed with respect to the continuum in zero. Two models for the lines required 

%matplotlib

### inline
for jj in [0]:  #  range(len(resultListDT)):
    plt.plot(Lambda[jj],Spectra[jj], lw=0.5, c = 'k', label='Spectra')
    plt.plot(Lambda[jj],SpectraCont[jj], lw=2, c = 'g', label='Spectra')

#co = 8
#1343	
#plt.plot(Lambda[jj][1339-co:1346+co],Spectra[jj][1339-co:1346+co], lw=0.5, c = 'k', label='Spectra')

#plt.plot(Lambda[jj],spec2[jj], lw=2, c = 'k', label='Spectra')

#plt.plot(Lambda[jj],SpectraCont[jj], lw=2, c='g', label='reducedSpectra')

Using matplotlib backend: Qt4Agg


###     EqWidth ,  FWHM,  FLUX

In [124]:
#   The plot of relative intensity and continuum emission for the galaxy index jj and the wavelenght of all the lines
#   The equivalent width, flux and FWHM for galaxy jj is measured too

#orderU = 10**(int(mt.log(np.mean(spectraFlux), 10))-1)

#spec =np.array(spectraFlux)
#spec2, lambd = r(spec, lambd, 0.004*orderU) # este parámetro de 0.004 puede ser igual para todas las galaxias
                                      #  pero seria bueno iterar sobre este valor para tener mejores continuos
                                      #  o tal vez tomar tres o maś regiones del espectro observado y calcular spec
                                    #  para diferentes parámetros de 0.1 a 0.001
        

#%matplotlib
for jj in range(len(resultListDT['ObjID'])-1):
    spe = []
    speCont = []
    for xind in range(len(Spectra[jj])):
        spe.append((Spectra[jj][xind])/(SpectraCont[jj][xind]))
        speCont.append((SpectraCont[jj][xind])/(SpectraCont[jj][xind]))
    #for i in range(len(Tablas[jj])):
    #    plt.axvline(x=Tablas[jj]['Lambda Vac'][i],c='dimgrey',lw=0.8,ls='--')
    #    plt.text(Tablas[jj]['Lambda Vac'][i] + 0.5, Tablas[jj]['Amplitude'][i] * 1.2,Tablas[jj]['Element'][i])

    #plt.plot(lambd, spe, c='k', lw=0.5, label='normalized data')
    #plt.plot(lambd, speCont, lw=2, label='normalized continuum')

    lFWHM = []
    eqW = []    ##  lista de valores 
    #  una vez normalizado el espectro observado a la medida del contínuo se integra por el método de Simpson
    for iLineProf in range(len(LINEProfile[jj])):
        f = FWHM(Lambda[jj][LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]], Spectra[jj][LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]])
        if len(f) == 0:
            lFWHM.append(0)
        else:
            lFWHM.append(f[0])
        sInteg = simps(y=spe[LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]], x=Lambda[jj][LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]], even='avg')
        eqW.append(sInteg)
    #   Flux determination for each model in galaxy list
    #   assignment of the FWHM and equivalent width in the Table for the first galaxy
    for i in range(len(Tablas[jj])):
        if Tablas[jj][i]['Amplitude']<=0:
            Tablas[jj]['Flux'][i],Tablas[jj]['Flux_Err'][i]= 0,0
        else:
            Tablas[jj]['Flux'][i],Tablas[jj]['Flux_Err'][i]=quad(gaussian,Tablas[jj][i]['Wavelength']-3*Tablas[jj][i]['Sigma'],Tablas[jj][i]['Wavelength']+3*Tablas[jj][i]['Sigma'],args=(Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Wavelength'],Tablas[jj][i]['Sigma']))
            Tablas[jj]['FWHM'][i],Tablas[jj]['eqWidth'][i] = lFWHM[i], eqW[i]
    Tablas[jj].write('Tabla'+str(resultListDT['ObjID'][jj])+'.tbl',format='ascii')
Tablas

[<Table masked=False length=15>
   Amplitude     Wavelength      Sigma      ...      FWHM        eqWidth   
    float64       float64       float64     ...    float64       float64   
 ------------- ------------- -------------- ... ------------- -------------
 23.2194792212 3727.02411685  1.33813822365 ...  5.1513671875 37.7133407593
 31.2626588832 3729.76488351  1.14203793096 ...  5.1513671875 37.7133407593
 12.3796863207 4341.52826646 0.812842687312 ... 2.99755859375 10.6157569885
 2.22288785359 4364.87897644  2.88642072881 ... 2.01025390625 4.98403072357
 25.9225838012 4862.52327468  1.31015672897 ...   3.361328125 18.8282089233
 15.8075300236 4945.19665676 0.306547181571 ...           0.0 4.86110401154
 15.2379012203 4960.40816747   1.9479364703 ... 5.70947265625 23.7358646393
 45.4847459477 5008.09877015  1.99648191965 ...    4.61328125 40.8398742676
           0.0       5756.24            0.0 ...           0.0           0.0
           0.0       6529.04            0.0 ...         

###        Electron density and temperature and plot

In [120]:
#   it obtains the electron density and temperature by the PyNeb cross iterative method for all couple of given species.

fig = plt.figure()
fig.suptitle('')
ax = fig.add_subplot(111)
ax.set_ylabel('Density $n_e (cm^{-3})$')
ax.set_xlabel('Temperature $T_e (K)$')

for jj in range(len(resultListDT)):
    k=0
    if Lambda[jj][0] > 3720.0 :
        k=2
    try:
        a=diags.getCrossTemDen('[OIII] 4363/5007+', '[SII] 6731/6716', Tablas[jj]['Flux'][3-k]/(Tablas[jj]['Flux'][6-k]+Tablas[jj]['Flux'][7-k]), Tablas[jj]['Flux'][13-k]/Tablas[jj]['Flux'][14-k])
        b=diags.getCrossTemDen('[NII] 5755/6584+', '[OII] 3726/3729', Tablas[jj]['Flux'][8-k]/(Tablas[jj]['Flux'][12-k]+Tablas[jj]['Flux'][10-k]), Tablas[jj]['Flux'][0-k]/Tablas[jj]['Flux'][1-k])
        c=diags.getCrossTemDen('[NII] 5755/6584+', '[SII] 6731/6716', Tablas[jj]['Flux'][8-k]/(Tablas[jj]['Flux'][12-k]+Tablas[jj]['Flux'][10-k]), Tablas[jj]['Flux'][13-k]/Tablas[jj]['Flux'][14-k])
        d=diags.getCrossTemDen('[OIII] 4363/5007+', '[OII] 3726/3729', Tablas[jj]['Flux'][3-k]/(Tablas[jj]['Flux'][6-k]+Tablas[jj]['Flux'][7-k]), Tablas[jj]['Flux'][0-k]/Tablas[jj]['Flux'][1-k])
    except ValueError:
        continue

    for i in IntrinsicHB:
        rc.setCorr((Tablas[jj]['Flux'][11-k]/Tablas[jj]['Flux'][4-k])/i, Tablas[jj]['Lambda Vac'][11-k], Tablas[jj]['Lambda Vac'][4-k])
        try:
            a2=diags.getCrossTemDen('[OIII] 4363/5007+', '[SII] 6731/6716',Tablas[jj]['Flux'][3-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][3-k])/(Tablas[jj]['Flux'][6-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][6-k])+ Tablas[jj]['Flux'][7-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][7-k]) ), Tablas[jj]['Flux'][13-k] * rc.getCorrHb(Tablas[jj]['Lambda Vac'][13-k])/Tablas[jj]['Flux'][14-k] * rc.getCorrHb(Tablas[jj]['Lambda Vac'][14-k]))
            b2=diags.getCrossTemDen('[NII] 5755/6584+', '[OII] 3726/3729', Tablas[jj]['Flux'][8-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][8-k])/(Tablas[jj]['Flux'][12-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][12-k])+Tablas[jj]['Flux'][10-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][7-k])), Tablas[jj]['Flux'][0-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][0-k])/Tablas[jj]['Flux'][1-k])*rc.getCorrHb(Tablas[jj]['Lambda Vac'][1-k])
            c2=diags.getCrossTemDen('[NII] 5755/6584+', '[SII] 6731/6716', Tablas[jj]['Flux'][8-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][8-k])/(Tablas[jj]['Flux'][12-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][12-k])+Tablas[jj]['Flux'][10-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][7-k])),Tablas[jj]['Flux'][13-k] * rc.getCorrHb(Tablas[jj]['Lambda Vac'][13-k])/Tablas[jj]['Flux'][14-k] * rc.getCorrHb(Tablas[jj]['Lambda Vac'][14-k]))
            d2=diags.getCrossTemDen('[OIII] 4363/5007+', '[OII] 3726/3729',Tablas[jj]['Flux'][3-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][3-k])/(Tablas[jj]['Flux'][6-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][6-k])+ Tablas[jj]['Flux'][7-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][7-k]) ), Tablas[jj]['Flux'][0-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][0-k])/Tablas[jj]['Flux'][1-k])*rc.getCorrHb(Tablas[jj]['Lambda Vac'][1-k])
        except ValueError:
            continue
        if b2 != () or d2 != ():
            plt.scatter(b2[0], b2[1],c='c')
            plt.scatter(d2[0], d2[1],c='c')
        if i == 2.86:
            resultListDT['Temp_1 [OIII]'][jj]=a2[0]
            resultListDT['Den_1 [SII]'][jj]=a2[1]
            resultListDT['Temp_3 [NII]'][jj]=c2[0]
            resultListDT['Den_3 [SII]'][jj]=c2[1]
            if b2 != () or d2 != ():
                resultListDT['Temp_2 [NII]'][jj]=b2[0]
                resultListDT['Den_2 [OII]'][jj]=b2[1] 
                resultListDT['Temp_4 [OIII]'][jj]=d2[0]
                resultListDT['Den_4 [OII]'][jj]=d2[1]
            if Lambda[jj][0] > 3720.0 :
                resultListDT['Temp_2 [NII]'][jj]=0.0
                resultListDT['Den_2 [OII]'][jj]=0.0
                resultListDT['Temp_4 [OIII]'][jj]=0.0
                resultListDT['Den_4 [OII]'][jj]=0.0
    resultListDT.write('TemDenGalaxies'+str(resultListDT['ObjID'][jj])+'.tbl',format='ascii')
print Tablas
resultListDT


[<Table masked=False length=15>
  Amplitude     Wavelength      Sigma      ...      FWHM        eqWidth   
   float64       float64       float64     ...    float64       float64   
------------- ------------- -------------- ... ------------- -------------
67.0461648916 3726.82943166  1.25192758822 ... 6.00805664062 34.9982833862
 67.998983131 3729.62879616  1.23957778677 ... 6.00805664062 34.9982833862
 26.509164056 4341.51295053  1.03200105682 ... 3.00146484375 10.1221055984
2.70926180733 4363.91155309 0.218998903387 ... 1.00244140625  2.1498477459
77.2745695529 4862.67220657  1.37292110209 ... 3.35693359375 21.1889533997
1.96403835508 4930.82344618   2.5568159224 ... 1.13818359375 3.65768780489
18.9178898933 4960.41726636  1.63526127448 ...  3.4287109375 16.9985389709
 57.852908807 5008.33584108   1.5644396663 ...  3.4619140625 21.7928305796
1.71319541219 5756.67337046  1.07765613729 ... 2.65087890625 4.13885232739
 1.5469530014 6529.08972551 0.297222979141 ... 1.50048828125 6.00106

ObjID,ra,dec,z,isFits,Temp_1 [OIII],Den_1 [SII],Temp_2 [NII],Den_2 [OII],Temp_3 [NII],Den_3 [SII],Temp_4 [OIII],Den_4 [OII]
,,,,,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3
string216,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64
SDSS J163620.40-000828.4SP2,249.085327148,-0.141222223639,0.087479,1,nan,nan,7752.74413695,2130.03013465,nan,nan,10921.7291419,2507.47932673
SDSS J164450.87+415547.2SP1,251.212005615,41.9297790527,0.096812,1,nan,nan,nan,nan,nan,nan,nan,nan
2MASX J16464090+2950063SP1,251.670578003,29.8351669312,0.062332,1,nan,nan,nan,nan,nan,nan,nan,nan
SDSS J165102.64+421314.1SP1,252.76133728,42.2206382751,0.253741,1,nan,nan,nan,nan,nan,nan,nan,nan
SDSS J165102.64+421314.1SP2,252.76133728,42.2206382751,0.253741,1,nan,nan,8376.1136269,4020.20145704,nan,nan,nan,nan
2MASX J16535206+6211495SP1,253.467163086,62.19713974,0.106069,1,nan,nan,10209.1972442,436.064384403,nan,nan,45764.2278122,762.575760346
2MASX J16535206+6211495SP2,253.467163086,62.19713974,0.106069,1,nan,nan,nan,nan,nan,nan,nan,nan


###                     Plots

In [24]:
#   plot of the galaxy with index jj observed with respect to the continuum in zero. Two models for the lines required 

%matplotlib
jj=0
if isFits:
    if TypeFITS == 0:
        print HDUs[jj][0].header['FLAVOR']
        print HDUs[jj][0].header['QUALITY']
        print HDUs[jj][0].header['FBADPIX'] ## %%Malos Pixeles
        print HDUs[jj][2].data['SN_MEDIAN_ALL']
        print HDUs[jj][2].data['SN_MEDIAN']

lambd=Lambda[jj]

plt.figure(figsize=(16, 9))
plt.title('Spectrum Galaxy '+ resultListDT['ObjID'][jj])
plt.xlabel(r'Wavelength (Angstroms $\AA$) ')
if TypeFITS == 0:
    plt.ylabel(r'$f_{\lambda}$ ('+str(head['BUNIT'])+')')
else:
    plt.ylabel(r'$f_{\lambda}$')

plt.errorbar(Lambda[jj],Spectra[jj]-SpectraCont[jj],SpectraIvar[jj],  c='gray',lw=1,label='Spectrum')   #Flujo en negro
#plt.errorbar(Lambda[jj],Spectra[jj]-SpectraCont[jj]-Modelos[jj][0](Lambda[jj]),SpectraIvar[jj],  c='k',lw=2,label='Data')   #Flujo en negro
#plt.errorbar(lambd,Spectra[jj] ,SpectraIvar[jj],  c='k',lw=2,label='Data')   #Flujo en negro
#plt.plot(lambd, spec2, lw=2, label='reducedSpectra')
#plt.plot(Lambda[jj], Modelos[jj][1](Lambda[jj]), c='cyan', lw=2, label='Model')
plt.plot(Lambda[jj], Spectra[jj]-SpectraCont[jj], c='k', lw=0.8, label='Model')
plt.plot(Lambda[jj], Modelos[jj][0](Lambda[jj]), c='yellow', lw=1)
#plt.plot(lambd, spec2, lw=2, label='reducedSpectra')

plt.legend()

x=np.linspace(Lambda[jj][0],Lambda[jj][-1],len(Lambda[jj]))
for i in range(len(Tablas[jj])):
    plt.axvline(x=Tablas[jj]['Lambda Vac'][i],c='dimgrey',lw=1.2,ls='--')
    plt.text(Tablas[jj]['Lambda Vac'][i] + 0.5, Tablas[jj]['Amplitude'][i] * 1.2,Tablas[jj]['Element'][i])
    if Tablas[jj]['Element'][i] == 'H I':
        plt.plot(x,gaussian(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Wavelength'],Tablas[jj][i]['Sigma']), c='DeepPink')
##        plt.plot(x,lorentzFunc(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Sigma'], Tablas[jj][i]['Wavelength']), c='DeepPink')
    else:
        if Tablas[jj][i]['Sigma']!=0:
            plt.plot(x,gaussian(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Wavelength'],Tablas[jj][i]['Sigma']) , c='Lime')
##            plt.plot(x,lorentzFunc(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Sigma'],Tablas[jj][i]['Wavelength']) , c='Lime')

#plt.savefig('F1.eps', format='eps', dpi=1000)


Using matplotlib backend: Qt4Agg


In [14]:
%matplotlib
kaksj = Modelos[jj][1](lambd)
lalskal = Modelos[jj][0](lambd)
specModel = spec+lalskal
plt.errorbar(lambd,Spectra[jj] ,SpectraIvar[jj],  c='k',lw=2,label='Data')   #Flujo en negro
plt.plot(lambd, specModel, lw=2, label='reducedSpectra')

Using matplotlib backend: Qt4Agg


In [14]:
%matplotlib

plt.figure(figsize=(16, 9))
for i in range(10):
    B=[]
    for j in range(len(Spectra[jj])):
        bb=SpectraIvar[jj][j]*random.uniform(-3,3)
        B.append(bb)
    plt.plot(lambd,(Spectra[jj] +B),c='orange')
plt.legend()

Using matplotlib backend: Qt4Agg


/usr/local/lib/python2.7/dist-packages/matplotlib/axes/_axes.py:475: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [11]:
Modelos[jj][0]

<CompoundModel13(amplitude_0=18.275806348814566, mean_0=3728.02822698702, stddev_0=1.9335914804280923, amplitude_1=8.855827600855736, mean_1=3731.3961514774714, stddev_1=0.8352339792949883, amplitude_2=1.4506676016571516, mean_2=4349.725289006149, stddev_2=3.230801297812224, amplitude_3=3.765523331518267, mean_3=4364.503840498161, stddev_3=4.621293961973244, amplitude_4=0.0, mean_4=4862.683, stddev_4=0.0, amplitude_5=2.1836195174164663, mean_5=4931.0478797037495, stddev_5=2.5023714309852845, amplitude_6=23.151990428591386, mean_6=4962.753863272741, stddev_6=4.305308949855951, amplitude_7=70.81786431320783, mean_7=5010.318496715861, stddev_7=3.8800202509568327, amplitude_8=2.7175063994114734, mean_8=5758.299602653674, stddev_8=-1.2780604468762424, amplitude_9=5.297654815010328, mean_9=6569.577899633828, stddev_9=-24.761413155863135, amplitude_10=9.662581981595594, mean_10=6549.6397884119915, stddev_10=2.3526062539049417, amplitude_11=11.529275992972986, mean_11=6565.392025105793, stddev

In [4]:
cooo

63

In [128]:
resultList = Table.read("resultList.tbl", format = 'ascii')
limDim = 1
aarr = [3, 6, 7, 10, 12, 13, 14, 0, 1]
for objID in resultList:
    nameTable = "Tabla"+str(objID[0])+".tbl"
    tableObjID = Table.read(nameTable, format = 'ascii')
    conta = 0
    for iLine in aarr: # range(len(tableObjID)):
        if (mt.fabs(tableObjID['Lambda Vac'][iLine]-tableObjID['Wavelength'][iLine]) > limDim) or (tableObjID['Flux'][iLine]==0.0) :
            continue
        else:
            conta = conta + 1
    if conta == len(aarr):
        print objID[0]

2MASX J00090793+1427558
SDSS J004119.73+002632.3
2MASX J00425255+1532465
SDSS J004335.93+151124.0
2MASX J00505919-0031294
MRK 1076SP2
MRK 0413SP1
MRK 0719SP1
MRK 0719SP2
MRK 0719SP4
MRK 0420SP1
MRK 0168SP1
UM 213SP1
SDSS J011031.04+010200.7SP1
MRK 1055SP1
SDSS J085450.07+324420.9SP1
MRK 0738SP1
TON 0151SP2
SDSS J131840.48+473425.2SP1
UM 653SP1
2MASX J15540451+2134564SP1
SDSS J162142.37+374536.7SP1
2MASX J16260432+4622060SP1
2MASX J00580076+0028456SP1
SDSS J020017.90+001429.8SP1
2MASX J03025710-0829425SP1
2MASX J08335433+5141416SP1
SDSS J083645.12+530235.7SP1
2MASX J09483005+5538228SP1
KUG 1016+375SP1
SDSS J115535.00-010615.7SP2
SDSS J150047.73+284746.5SP1
VV 705 NED01SP1
SDSS J161732.15-001701.6SP1
SDSS J163620.40-000828.4SP2
2MASX J16535206+6211495SP1
SDSS J170918.48+621731.5SP1
SDSS J170918.48+621731.5SP2
SDSS J171217.99+645906.7SP1
